In [23]:
import glob
import codecs
import nltk
import re
from __future__ import absolute_import, division, print_function
import pprint
from nltk.corpus import stopwords
from many_stop_words import get_stop_words

In [25]:
books_file_paths = sorted (glob.glob("./word_vectors_game_of_thrones-LIVE/data/*"))

# Show Different Stopwords
nltk_words = list(stopwords.words('english'))
manystopwords = list(get_stop_words('en'))

In [26]:
def rawBook_to_processed(abook):
    rawbookstring = u''
    with codecs.open(abook, "r", "utf-8") as book_file:
        rawbookstring += book_file.read()
    tokenizer = nltk.data.load('/Users/Zhanik/nltk_data/tokenizers/punkt/english.pickle')
    
    
    raw_sentences = tokenizer.tokenize(rawbookstring)
    
    nltk_words = list(stopwords.words('english'))
    manystopwords = list(get_stop_words('en')) 
    def sentence_to_wordlist(raw):
        clean = re.sub("[^a-zA-Z]"," ", raw)
        words = clean.split()
        lowercase = [word.strip().lower() for word in words]
        filtered_word_list = lowercase[:]  # make a copy of the word_list
        
        for word in lowercase:  # iterate over word_list
            if word in manystopwords: 
                filtered_word_list.remove(word) #
        return filtered_word_list
    
    words = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            words.append(sentence_to_wordlist(raw_sentence))
    return words


In [27]:
words1 = rawBook_to_processed(books_file_paths[0])

In [28]:
flat_list = [item for sublist in words1 for item in sublist]

In [29]:
books = []

In [30]:
for afile in books_file_paths:
    books.append ( [item.lower() for sublist in rawBook_to_processed(afile) for item in sublist] )

In [31]:
len(books)

5

In [32]:
from gensim import corpora

In [33]:
dictionary = corpora.Dictionary(books)
dictionary.save('./LDApickles/Got.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(22296 unique tokens: [u'fawn', u'mustachio', u'woods', u'clotted', u'spiders']...)


In [11]:
print(dictionary.token2id)

{u'fawn': 0, u'mustachio': 16995, u'woods': 6, u'clotted': 7, u'spiders': 8, u'hanging': 1806, u'woody': 21817, u'sevens': 12826, u'lewyn': 12828, u'dragonbone': 18, u'scold': 19, u'clanked': 4487, u'caned': 23, u'unnecessarily': 22471, u'crossbar': 21832, u'lewys': 122, u'wracked': 17009, u'donnelwood': 13068, u'bringing': 36, u'wooded': 37, u'grueling': 12862, u'wooden': 39, u'crotch': 43, u'pisspot': 12591, u'festered': 46, u'guardsmen': 48, u'sooty': 50, u'pizzle': 20282, u'lefford': 53, u'scrapes': 17030, u'scraped': 63, u'snuggled': 65, u'errors': 17040, u'harsley': 12898, u'donyse': 12899, u'deferred': 19192, u'cooking': 69, u'pawed': 71, u'succumb': 12910, u'shocks': 12912, u'crouch': 12914, u'chins': 77, u'quagmires': 12922, u'chink': 85, u'natured': 86, u'kids': 18980, u'yronwoods': 20295, u'outdrink': 12935, u'climbed': 89, u'spotty': 12946, u'climber': 93, u'golden': 94, u'cobblers': 20299, u'kinslayers': 95, u'lengthen': 98, u'stern': 101, u'quentyn': 102, u'forbore': 2030

In [13]:
print(dictionary.doc2bow(books[0]))

[(0, 4), (1, 3), (2, 1), (3, 12), (4, 94), (5, 1), (6, 28), (7, 1), (8, 2), (9, 13), (10, 1), (11, 1), (12, 16), (13, 1), (14, 1), (15, 1), (16, 3), (17, 2), (18, 3), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1516), (26, 1), (27, 5), (28, 8), (29, 7), (30, 5), (31, 3), (32, 54), (33, 2), (34, 51), (35, 9), (36, 22), (37, 3), (38, 3), (39, 60), (40, 4), (41, 4), (42, 2), (43, 1), (44, 8), (45, 4), (46, 2), (47, 1), (48, 23), (49, 2), (50, 2), (51, 3), (52, 22), (53, 5), (54, 1), (55, 1), (56, 15), (57, 1), (58, 5), (59, 66), (60, 3), (61, 8), (62, 22), (63, 3), (64, 1), (65, 1), (66, 3), (67, 4), (68, 7), (69, 6), (70, 178), (71, 2), (72, 8), (73, 3), (74, 6), (75, 7), (76, 1), (77, 1), (78, 1), (79, 4), (80, 3), (81, 7), (82, 15), (83, 2), (84, 5), (85, 1), (86, 1), (87, 6), (88, 2), (89, 54), (90, 4), (91, 4), (92, 1), (93, 2), (94, 98), (95, 1), (96, 3), (97, 12), (98, 1), (99, 4), (100, 148), (101, 12), (102, 1), (103, 1), (104, 1), (105, 53), (106, 1), (107, 1), (1

In [34]:
corpus = [dictionary.doc2bow(text) for text in books]

In [35]:
import gensim.models.ldamodel as lda

In [36]:
mymodel = lda.LdaModel(corpus, id2word=dictionary, num_topics=100)

In [38]:
var = mymodel.print_topic(88)

In [37]:
mymodel.print_topic(10, topn=100)

u'0.012*"lord" + 0.011*"ser" + 0.006*"man" + 0.006*"king" + 0.005*"jon" + 0.005*"men" + 0.005*"tyrion" + 0.004*"lady" + 0.004*"father" + 0.004*"black" + 0.004*"long" + 0.004*"ll" + 0.003*"time" + 0.003*"hand" + 0.003*"face" + 0.003*"told" + 0.003*"boy" + 0.003*"son" + 0.003*"head" + 0.003*"thought" + 0.003*"white" + 0.003*"night" + 0.003*"eyes" + 0.003*"sansa" + 0.003*"queen" + 0.003*"good" + 0.002*"red" + 0.002*"brother" + 0.002*"wall" + 0.002*"heard" + 0.002*"hair" + 0.002*"girl" + 0.002*"sword" + 0.002*"maester" + 0.002*"cersei" + 0.002*"great" + 0.002*"jaime" + 0.002*"day" + 0.002*"hands" + 0.002*"knew" + 0.002*"called" + 0.002*"blood" + 0.002*"arya" + 0.002*"stark" + 0.002*"snow" + 0.002*"grace" + 0.002*"high" + 0.002*"stone" + 0.002*"prince" + 0.002*"robb" + 0.002*"bran" + 0.002*"castle" + 0.002*"looked" + 0.002*"woman" + 0.002*"water" + 0.002*"dead" + 0.002*"gods" + 0.002*"turned" + 0.002*"winterfell" + 0.002*"iron" + 0.002*"gold" + 0.002*"catelyn" + 0.002*"house" + 0.002*"grey"

In [39]:
var

u'0.010*"ser" + 0.009*"lord" + 0.006*"king" + 0.006*"man" + 0.006*"father" + 0.005*"men" + 0.005*"jon" + 0.004*"tyrion" + 0.004*"thought" + 0.004*"black"'

In [40]:
var.encode('ascii','replace')

'0.010*"ser" + 0.009*"lord" + 0.006*"king" + 0.006*"man" + 0.006*"father" + 0.005*"men" + 0.005*"jon" + 0.004*"tyrion" + 0.004*"thought" + 0.004*"black"'